In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [78]:
import pickle
with open('raw_data_20180811.pickle','rb') as handle:

    raw_data = pickle.load(handle)

In [79]:
kospi200 = pd.read_excel("C:\\Users\\axasd\\Downloads\\상장법인목록.xlsx")
kospi200 = kospi200[kospi200.columns[:2]]
kospi200["종목코드"] = kospi200["종목코드"].map('{:06d}'.format)

In [80]:
code_dict = {
    i : j for i, j in zip(kospi200["회사명"], kospi200["종목코드"])
}

In [81]:
for i in raw_data.keys():
    raw_data[i] = raw_data[i].assign(sum_close_3 = raw_data[i].Close.rolling(window = 3).sum())
    raw_data[i] = raw_data[i].assign(sum_close_5 = raw_data[i].Close.rolling(window =5).sum())
    raw_data[i] = raw_data[i].assign(ma_5 = raw_data[i].Close.rolling(window = 5).mean())
    raw_data[i] = raw_data[i].assign(ma_10 = raw_data[i].Close.rolling(window = 10).mean())
    
for i in raw_data.keys():
    raw_data[i] = raw_data[i].assign(pct_change = raw_data[i].Close.pct_change().fillna(0))
    raw_data[i] = raw_data[i].assign(Y = np.where(raw_data[i]["pct_change"].fillna(0)>=0, 1,0))
    
for i in raw_data.keys():
    raw_data[i] = raw_data[i].assign(pct_change2 = raw_data[i].sum_close_3.pct_change().fillna(0))
    raw_data[i] = raw_data[i].assign(pct_change3 = raw_data[i].sum_close_5.pct_change().fillna(0))
    raw_data[i] = raw_data[i].assign(sencond_Y = np.where(raw_data[i]["pct_change2"].fillna(0)>=0, 1,0))
    raw_data[i] = raw_data[i].assign(third_Y = np.where(raw_data[i]["pct_change3"].fillna(0)>=0, 1,0))
    
for i in raw_data.keys():
    raw_data[i] = raw_data[i].dropna(axis=0)

In [86]:
copy_keys = kospi200["종목코드"]
for i in copy_keys:
    if (len(raw_data[i]) < 252):
        del raw_data[i]

### Train, Test Split

In [87]:
train1 = {}
train_idx1 = {}
test1 = {}
test_idx1 = {}
for code, df in raw_data.items():
    train1[code] = df.iloc[:int(len(df)*0.8)][["Open"]]
    train_idx1[code] = df.iloc[:int(len(df)*0.8)]["third_Y"]
    test1[code] = df.iloc[int(len(df)*0.8):][["Open"]]
    test_idx1[code] = df.iloc[int(len(df)*0.8):]["third_Y"]

In [88]:
train2 = {}
train_idx2 = {}
test2 = {}
test_idx2 = {}
for code, df in raw_data.items():
    train2[code] = df.iloc[:int(len(df)*0.8)][["ma_5"]]
    train_idx2[code] = df.iloc[:int(len(df)*0.8)]["third_Y"]
    test2[code] = df.iloc[int(len(df)*0.8):][["ma_5"]]
    test_idx2[code] = df.iloc[int(len(df)*0.8):]["third_Y"]

In [89]:
train3 = {}
train_idx3 = {}
test3 = {}
test_idx3 = {}
for code, df in raw_data.items():
    train3[code] = df.iloc[:int(len(df)*0.8)][["Open","ma_5"]]
    train_idx3[code] = df.iloc[:int(len(df)*0.8)]["third_Y"]
    test3[code] = df.iloc[int(len(df)*0.8):][["Open","ma_5"]]
    test_idx3[code] = df.iloc[int(len(df)*0.8):]["third_Y"]

In [105]:
train4 = {}
train_idx4 = {}
test4 = {}
test_idx4 = {}
for code, df in raw_data.items():
    train4[code] = df.iloc[:int(len(df)*0.8)][["ma_5","ma_10"]]
    train_idx4[code] = df.iloc[:int(len(df)*0.8)]["third_Y"]
    test4[code] = df.iloc[int(len(df)*0.8):][["ma_5","ma_10"]]
    test_idx4[code] = df.iloc[int(len(df)*0.8):]["third_Y"]

In [110]:
train5 = {}
train_idx5 = {}
test5 = {}
test_idx5 = {}
for code, df in raw_data.items():
    train5[code] = df.iloc[:int(len(df)*0.8)][["Open","ma_5","ma_10"]]
    train_idx5[code] = df.iloc[:int(len(df)*0.8)]["third_Y"]
    test5[code] = df.iloc[int(len(df)*0.8):][["Open","ma_5","ma_10"]]
    test_idx5[code] = df.iloc[int(len(df)*0.8):]["third_Y"]

### Scale 조정

In [90]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
li1 = ["Open"]
for code in raw_data.keys():
    train1[code][li1] = std_scaler.fit_transform(train1[code][li1].values)
    test1[code][li1] = std_scaler.fit_transform(test1[code][li1].values)

In [91]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
li2 = ["ma_5"]
for code in raw_data.keys():
    train2[code][li2] = std_scaler.fit_transform(train2[code][li2].values)
    test2[code][li2] = std_scaler.fit_transform(test2[code][li2].values)

In [92]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
li3 = ["Open","ma_5"]
for code in raw_data.keys():
    train3[code][li3] = std_scaler.fit_transform(train3[code][li3].values)
    test3[code][li3] = std_scaler.fit_transform(test3[code][li3].values)

In [106]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
li4 = ["ma_5","ma_10"]
for code in raw_data.keys():
    train4[code][li4] = std_scaler.fit_transform(train4[code][li4].values)
    test4[code][li4] = std_scaler.fit_transform(test4[code][li4].values)

In [111]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
li5 = ["Open","ma_5","ma_10"]
for code in raw_data.keys():
    train5[code][li5] = std_scaler.fit_transform(train5[code][li5].values)
    test5[code][li5] = std_scaler.fit_transform(test5[code][li5].values)

### model 호출

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

log_clf = LogisticRegression()
DTC_clf = DecisionTreeClassifier()
svm_clf = SVC()

### Predict

In [95]:
from sklearn.metrics import accuracy_score

score_list_1 = pd.DataFrame(columns=('LR','DT','SVM'))

for i in raw_data.keys():
    a1=[]
    X_train1 = train1[i].values
    y_train1 = train_idx1[i].values
    X_test1 = test1[i].values
    y_test1 = test_idx1[i].values
    for clf in (log_clf, DTC_clf, svm_clf):
        clf.fit(X_train1, y_train1)
        y_pred1 = clf.predict(X_test1)
        a1.append(accuracy_score(y_test1, y_pred1))
    score_list_1.loc[i] = a1

In [99]:
from sklearn.metrics import accuracy_score

score_list_2 = pd.DataFrame(columns=('LR','DT','SVM'))

for i in raw_data.keys():
    a2=[]
    X_train2 = train2[i].values
    y_train2 = train_idx2[i].values
    X_test2 = test2[i].values
    y_test2 = test_idx2[i].values
    for clf in (log_clf, DTC_clf, svm_clf):
        clf.fit(X_train2, y_train2)
        y_pred2 = clf.predict(X_test2)
        a2.append(accuracy_score(y_test2, y_pred2))
    score_list_2.loc[i] = a2

In [100]:
from sklearn.metrics import accuracy_score

score_list_3 = pd.DataFrame(columns=('LR','DT','SVM'))

for i in raw_data.keys():
    a3=[]
    X_train3 = train3[i].values
    y_train3 = train_idx3[i].values
    X_test3 = test3[i].values
    y_test3 = test_idx3[i].values
    for clf in (log_clf, DTC_clf, svm_clf):
        clf.fit(X_train3, y_train3)
        y_pred3 = clf.predict(X_test3)
        a3.append(accuracy_score(y_test3, y_pred3))
    score_list_3.loc[i] = a3

In [107]:
from sklearn.metrics import accuracy_score

score_list_4 = pd.DataFrame(columns=('LR','DT','SVM'))

for i in raw_data.keys():
    a4=[]
    X_train4 = train4[i].values
    y_train4 = train_idx4[i].values
    X_test4 = test4[i].values
    y_test4 = test_idx4[i].values
    for clf in (log_clf, DTC_clf, svm_clf):
        clf.fit(X_train4, y_train4)
        y_pred4 = clf.predict(X_test4)
        a4.append(accuracy_score(y_test4, y_pred4))
    score_list_4.loc[i] = a4

In [112]:
from sklearn.metrics import accuracy_score

score_list_5 = pd.DataFrame(columns=('LR','DT','SVM'))

for i in raw_data.keys():
    a5=[]
    X_train5 = train5[i].values
    y_train5 = train_idx5[i].values
    X_test5 = test5[i].values
    y_test5 = test_idx5[i].values
    for clf in (log_clf, DTC_clf, svm_clf):
        clf.fit(X_train5, y_train5)
        y_pred5 = clf.predict(X_test5)
        a5.append(accuracy_score(y_test5, y_pred5))
    score_list_5.loc[i] = a5

## 결과

In [113]:
print('feature : open')
print(score_list_1.describe())
print('feature : MA5')
print(score_list_2.describe())
print('feature : open, MA5')
print(score_list_3.describe())
print('feature : MA5, MA10')
print(score_list_4.describe())
print('feature : open,MA5, MA10')
print(score_list_5.describe())

feature : open
               LR          DT         SVM
count  197.000000  197.000000  197.000000
mean     0.518685    0.505691    0.512399
std      0.052390    0.037905    0.046066
min      0.365325    0.300310    0.386997
25%      0.482972    0.486068    0.486068
50%      0.523220    0.507740    0.510836
75%      0.554180    0.529412    0.541796
max      0.715170    0.606811    0.671827
feature : MA5
               LR          DT         SVM
count  197.000000  197.000000  197.000000
mean     0.497324    0.501337    0.501036
std      0.048207    0.031478    0.043638
min      0.291022    0.391753    0.386997
25%      0.467492    0.480702    0.473684
50%      0.497326    0.501548    0.504644
75%      0.530769    0.520124    0.526316
max      0.656934    0.702786    0.687307
feature : open, MA5
               LR          DT         SVM
count  197.000000  197.000000  197.000000
mean     0.678308    0.611466    0.632371
std      0.042324    0.036300    0.066762
min      0.463415    0.4953

## MA간의 비교를 할때, 모델성능이 크게 좋아졌으며, open과 MA간의 비교또한 모델성능을 향상시키는데 유의미한 요소였다. 